# E-Commerce Sales Intelligence System  
## 02 — Data Cleaning & Preparation


In [1]:
import pandas as pd
import numpy as np

archive = "archive/"

customers = pd.read_csv(archive + "olist_customers_dataset.csv")
geolocation = pd.read_csv(archive + "olist_geolocation_dataset.csv")
orders = pd.read_csv(archive + "olist_orders_dataset.csv")
order_items = pd.read_csv(archive + "olist_order_items_dataset.csv")
order_payments = pd.read_csv(archive + "olist_order_payments_dataset.csv")
order_reviews = pd.read_csv(archive + "olist_order_reviews_dataset.csv")
products = pd.read_csv(archive + "olist_products_dataset.csv")
sellers = pd.read_csv(archive + "olist_sellers_dataset.csv")
product_translation = pd.read_csv(archive + "product_category_name_translation.csv")


### Orders table

In [2]:
orders.info()
orders.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

#### Convert date columns to datetime

In [3]:
date_cols = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]

for col in date_cols:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')


#### Understand missing values meaning

#### Create a Delivery Status Flag

In [4]:
orders["delivered_flag"] = orders["order_delivered_customer_date"].notna().astype(int)


#### Filter to only “completed/delivered” orders for revenue analysis

In [5]:
delivered_orders = orders[orders["order_status"] == "delivered"].copy()


#### Handle cancellations

In [6]:
cancelled_orders = orders[orders["order_status"] == "canceled"]


#### Check delivery delays

In [7]:
orders["delivery_time_days"] = (
    orders["order_delivered_customer_date"] - orders["order_purchase_timestamp"]
).dt.days


#### Estimate delay vs estimated

In [8]:
orders["estimated_time_days"] = (
    orders["order_estimated_delivery_date"] - orders["order_purchase_timestamp"]
).dt.days


In [9]:
orders["delay_days"] = orders["delivery_time_days"] - orders["estimated_time_days"]


### Order_items Table

In [11]:
order_items.info()
order_items.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

#### Convert shipping_limit_date to datetime

In [12]:
order_items["shipping_limit_date"] = pd.to_datetime(
    order_items["shipping_limit_date"], 
    errors='coerce'
)


#### Create Total Item Value (Price + Freight)

In [13]:
order_items["total_item_value"] = order_items["price"] + order_items["freight_value"]


#### Calculate revenue per order

In [14]:
order_revenue = (
    order_items.groupby("order_id")["total_item_value"]
    .sum()
    .reset_index()
    .rename(columns={"total_item_value": "order_revenue"})
)


### Product Table

In [16]:
products.info()
products.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

#### Merge product category translation

In [29]:
products = products.merge(
    product_translation,
    on="product_category_name",
    how="left"
)
# Because category names are Portuguese, and BI dashboards need English.
# Run this once to avoid duplicacy of columns

#### Handle missing product_category_name

In [32]:
products.columns

Index(['product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'product_category_name_english'],
      dtype='object')

In [31]:
products["product_category_name"].fillna("unknown_category", inplace=True)
products["product_category_name_english"].fillna("Unknown", inplace=True)


### Start Building the MASTER DATASET

### Merge orders + order items

In [33]:
order_items_merged = orders.merge(
    order_items,
    on="order_id",
    how="inner"
)
order_items_merged.head()
# This gives you a row for every item purchased.

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivered_flag,delivery_time_days,estimated_time_days,delay_days,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,total_item_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,1,8.0,15,-7.0,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,38.71
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,1,13.0,19,-6.0,1,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,141.46
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,1,9.0,26,-17.0,1,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,179.12
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,1,13.0,26,-13.0,1,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20,72.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,1,2.0,12,-10.0,1,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72,28.62


### Then merge products

In [34]:
order_items_products = order_items_merged.merge(
    products,
    on="product_id",
    how="left"
)
order_items_products.head()


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivered_flag,delivery_time_days,...,total_item_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,1,8.0,...,38.71,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,1,13.0,...,141.46,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0,perfumery
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,1,9.0,...,179.12,automotivo,46.0,232.0,1.0,420.0,24.0,19.0,21.0,auto
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,1,13.0,...,72.20,pet_shop,59.0,468.0,3.0,450.0,30.0,10.0,20.0,pet_shop
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,1,2.0,...,28.62,papelaria,38.0,316.0,4.0,250.0,51.0,15.0,15.0,stationery


### Then merge customers

In [35]:
full_data = order_items_products.merge(
    customers,
    on="customer_id",
    how="left"
)


### Then merge sellers

In [37]:
full_data = full_data.merge(
    sellers,
    on="seller_id",
    how="left"
)


In [38]:
order_items_merged = orders.merge(order_items, on="order_id", how="inner")


### Clean & Enhance the MASTER DATASET

#### Convert date columns to datetime

In [40]:
date_cols = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date",
    "shipping_limit_date"
]

for col in date_cols:
    full_data[col] = pd.to_datetime(full_data[col], errors="coerce")


#### Create revenue & cost metrics

#### Actual revenue for each line item

In [42]:
full_data["item_revenue"] = full_data["price"]


#### Total cost of shipping paid by the customer

In [43]:
full_data["shipping_cost"] = full_data["freight_value"]


#### Delivery speed metrics

#### Number of days between purchase and customer delivery

In [44]:
full_data["delivery_days"] = (
    full_data["order_delivered_customer_date"] - full_data["order_purchase_timestamp"]
).dt.days


#### Number of days delayed (if delivered later than estimated)

In [45]:
full_data["delivery_delay"] = (
    full_data["order_delivered_customer_date"] - full_data["order_estimated_delivery_date"]
).dt.days


#### Filter to only delivered orders

In [46]:
full_data = full_data[full_data["order_status"] == "delivered"]


In [47]:
full_data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 110197 entries, 0 to 112649
Data columns (total 39 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       110197 non-null  object        
 1   customer_id                    110197 non-null  object        
 2   order_status                   110197 non-null  object        
 3   order_purchase_timestamp       110197 non-null  datetime64[ns]
 4   order_approved_at              110182 non-null  datetime64[ns]
 5   order_delivered_carrier_date   110195 non-null  datetime64[ns]
 6   order_delivered_customer_date  110189 non-null  datetime64[ns]
 7   order_estimated_delivery_date  110197 non-null  datetime64[ns]
 8   delivered_flag                 110197 non-null  int32         
 9   delivery_time_days             110189 non-null  float64       
 10  estimated_time_days            110197 non-null  int64         
 11  

### Exploratory Data Analysis

### Basic KPIs

#### Total Revenue

In [48]:
total_revenue = full_data["item_revenue"].sum()
total_revenue


13221498.110000001

#### Total Orders

In [49]:
total_orders = full_data["order_id"].nunique()
total_orders


96478

#### Total Customers

In [50]:
total_customers = full_data["customer_unique_id"].nunique()
total_customers


93358

#### Total Sellers

In [51]:
total_sellers = full_data["seller_id"].nunique()
total_sellers


2970

### Category-wise Revenue

In [52]:
category_revenue = (
    full_data.groupby("product_category_name_english")["item_revenue"]
    .sum()
    .sort_values(ascending=False)
)
category_revenue.head(10)


product_category_name_english
health_beauty            1233131.72
watches_gifts            1166176.98
bed_bath_table           1023434.76
sports_leisure            954852.55
computers_accessories     888724.61
furniture_decor           711927.69
housewares                615628.69
cool_stuff                610204.10
auto                      578966.65
toys                      471286.48
Name: item_revenue, dtype: float64

### Monthly Sales Trend

In [53]:
full_data["order_month"] = full_data["order_purchase_timestamp"].dt.to_period("M")

monthly_sales = (
    full_data.groupby("order_month")["item_revenue"]
    .sum()
    .reset_index()
)
monthly_sales.head()


,order_month,item_revenue
0,2016-09,134.97
1,2016-10,40325.11
2,2016-12,10.90
3,2017-01,111798.36
4,2017-02,234223.40


### Delivery Performance

#### Average delivery days:

In [54]:
full_data["delivery_days"].mean()


12.007341930682736

#### Average delay:

In [58]:
full_data["delivery_delay"].mean()


-12.02904101135322

1. Monthly Revenue Analysis
2. Delivery Time Analysis
3. Customer Segmentation (RFM)
4. Cohort Retention Analysis
5. Payment Analytics
6. Product-Level Deep Dive
7. Seller Analysis
8. Geo Analysis
9. Review Score Analysis
10. Predictive Modeling

## Exporting Dataset

In [62]:
# Check your variable name
%whos


Variable               Type         Data/Info
---------------------------------------------
archive                str          archive/
cancelled_orders       DataFrame                             <...>n\n[625 rows x 9 columns]
category_revenue       Series       product_category_name_eng<...>ength: 72, dtype: float64
col                    str          shipping_limit_date
customers              DataFrame                             <...>n[99441 rows x 5 columns]
date_cols              list         n=6
delivered_orders       DataFrame                             <...>n[96478 rows x 9 columns]
full_data              DataFrame                             <...>110197 rows x 40 columns]
geolocation            DataFrame             geolocation_zip_<...>1000163 rows x 5 columns]
monthly_sales          DataFrame       order_month  item_reve<...>    2018-08     838576.64
np                     module       <module 'numpy' from 'C:\<...>ges\\numpy\\__init__.py'>
order_items            DataFram

In [63]:
full_data.to_csv("final_olist_dataset.csv", index=False)
